## METADATA For the Spaced Repitition Dataset 
* p_recall - proportion of exercises from this lesson/practice where the word/lexeme was correctly recalled
* timestamp - UNIX timestamp of the current lesson/practice
* delta - time (in seconds) since the last lesson/practice that included this word/lexeme
* user_id - student user ID who did the lesson/practice (anonymized)
learning_language - language being learned
* ui_language - user interface language (presumably native to the student)
* lexeme_id - system ID for the lexeme tag (i.e., word)
* lexeme_string - lexeme tag (see below)
* history_seen - total times user has seen the word/lexeme prior to this lesson/practice
* history_correct - total times user has been correct for the word/lexeme prior to this lesson/practice
* session_seen - times the user saw the word/lexeme during this lesson/practice
* session_correct - times the user got the word/lexeme correct during this lesson/practice

In [1]:
#installing packages 
!pip install plotly

In [2]:
pip install path

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install parquet 

Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing libraries
from pathlib import Path
import pandas as pd
import numpy as np
import zipfile
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import gzip
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from zipfile import ZipFile 
%matplotlib inline

In [3]:
# Load packages
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    RocCurveDisplay,
)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import utils

In [4]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
## data acquisition 
## SPACED REPETITION DATA 
dataframe = pd.read_csv("/Users/zeinebouerghi/Downloads/duolingospacedrd.csv.gz")
dataframe.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1


In [8]:
dataframe.describe()

,p_recall,timestamp,delta,history_seen,history_correct,session_seen,session_correct
count,1.285423e+07,1.285423e+07,1.285423e+07,1.285423e+07,1.285423e+07,1.285423e+07,1.285423e+07
mean,8.961056e-01,1.362589e+09,7.295811e+05,2.198109e+01,1.935025e+01,1.817686e+00,1.644134e+00
std,2.714048e-01,2.932082e+05,2.246499e+06,1.295508e+02,1.119681e+02,1.360180e+00,1.318794e+00
min,0.000000e+00,1.362076e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,1.362343e+09,5.320000e+02,3.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.362591e+09,7.713400e+04,6.000000e+00,6.000000e+00,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.362846e+09,4.425070e+05,1.500000e+01,1.300000e+01,2.000000e+00,2.000000e+00
max,1.000000e+00,1.363105e+09,4.032836e+07,1.351800e+04,1.288800e+04,2.000000e+01,2.000000e+01


In [9]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12854226 entries, 0 to 12854225
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   p_recall           float64
 1   timestamp          int64  
 2   delta              int64  
 3   user_id            object 
 4   learning_language  object 
 5   ui_language        object 
 6   lexeme_id          object 
 7   lexeme_string      object 
 8   history_seen       int64  
 9   history_correct    int64  
 10  session_seen       int64  
 11  session_correct    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 1.1+ GB


In [10]:
correlation_data = dataframe.corr()

In [11]:
correlation_data.style.background_gradient()

,p_recall,timestamp,delta,history_seen,history_correct,session_seen,session_correct
p_recall,1.000000,-0.000709,-0.030221,-0.022747,-0.012844,0.041433,0.301793
timestamp,-0.000709,1.000000,0.024238,-0.002188,-0.003297,0.010319,0.009683
delta,-0.030221,0.024238,1.000000,-0.030382,-0.030217,0.002170,-0.006933
history_seen,-0.022747,-0.002188,-0.030382,1.000000,0.985646,0.002899,-0.004417
history_correct,-0.012844,-0.003297,-0.030217,0.985646,1.000000,0.003628,-0.000407
session_seen,0.041433,0.010319,0.002170,0.002899,0.003628,1.000000,0.952811
session_correct,0.301793,0.009683,-0.006933,-0.004417,-0.000407,0.952811,1.000000


In [12]:
data_dir = Path('/Users/zeinebouerghi/Desktop/dataverse_files')

In [10]:
df = pd.read_parquet('/Users/zeinebouerghi/Desktop/dataverse_files/testpart1.parquet')

In [12]:
df.head()

,datetime,ui_language,eligible_templates,history,selected_template,session_end_completed
0,18.541030,ru,"[K, H, G, E, B, J, L, F, D, A]",None,L,False
1,21.052685,dn,"[K, H, G, E, B, J, L, F, D]",None,H,False
2,31.855683,es,"[K, H, G, E, B, J, L, F, D]",None,F,False
3,19.486713,pt,"[K, H, G, E, B, J, L, F, D, A]",None,L,False
4,22.296007,pt,"[K, H, G, E, B, J, L, F, D, A]",None,D,True


In [13]:
df.history.unique()

array([None], dtype=object)

In [14]:
df.describe()

,datetime
count,5.212425e+07
mean,2.416318e+01
std,5.172619e+00
min,1.500000e+01
25%,1.965360e+01
50%,2.430299e+01
75%,2.866520e+01
max,3.284222e+01


In [15]:
# get the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52124250 entries, 0 to 52124249
Data columns (total 6 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   datetime               float64
 1   ui_language            object 
 2   eligible_templates     object 
 3   history                object 
 4   selected_template      object 
 5   session_end_completed  bool   
dtypes: bool(1), float64(1), object(4)
memory usage: 2.0+ GB


In [16]:
#interested in the unique number of values of the sessions completed
df.session_end_completed.nunique()

2

In [17]:
# Check if there are any null values
print(df.isnull().sum())

datetime                        0
ui_language                     0
eligible_templates              0
history                  52124250
selected_template               0
session_end_completed           0
dtype: int64


In [18]:
print(dataframe.shape)

(12854226, 12)


In [13]:
#target variable is the number of recall times 
y = dataframe["p_recall"] 

In [14]:
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

In [15]:
y.head()

0    1.0
1    0.5
2    1.0
3    0.5
4    1.0
Name: p_recall, dtype: float64

In [16]:
#y_transformed.head()

In [17]:
Y = dataframe.iloc[:, 0:2]
Y.head()

,p_recall,timestamp
0,1.0,1362076081
1,0.5,1362076081
2,1.0,1362076081
3,0.5,1362076081
4,1.0,1362076081


In [18]:
#fitting the features, we will look into the history of the mistakes based on the number of times it was seen
X = dataframe.iloc[:, 8:14]
X.head()

,history_seen,history_correct,session_seen,session_correct
0,6,4,2,2
1,4,4,2,1
2,5,4,1,1
3,6,5,2,1
4,4,4,1,1


In [19]:
# Split the data into train and test subsets
# You can adjust the test size and random state
X_train, X_test, y_train, y_test = train_test_split(
    X, y_transformed, test_size=0.30, random_state=123)

# Standardize X data based on X_train
sc = StandardScaler().fit(X_train)
X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

In [ ]:
model = LogisticRegression(random_state = 10)
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
# Define parameters: these will need to be tuned to prevent overfitting and underfitting
params = {
    "penalty": "l2",  # Norm of the penalty: 'l1', 'l2', 'elasticnet', 'none'
    "C": 1,  # Inverse of regularization strength, a positive float
    "random_state": 123,
}

# Create a logistic regression classifier object with the parameters above
clf = LogisticRegression(**params)

# Train the classifer on the train set
clf = clf.fit(X_train_scaled, y_train)

# Predict the outcomes on the test set
y_pred = clf.predict(X_test_scaled)

In [ ]:
#evaluating the classifier through the accuracy, precision, and recall scores 
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

In [ ]:
# Calculate confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

# Plot a labeled confusion matrix with Seaborn
sns.heatmap(cnf_matrix, annot=True, fmt="g")
plt.title("Confusion matrix")
plt.ylabel("Actual label")
plt.xlabel("Predicted label")

In [ ]:
# Plot ROC curve
RocCurveDisplay.from_estimator(clf, X_test_scaled, y_test)

### References:
Data can be downloaded here :
***
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/23ZWVI
@data{DVN/23ZWVI_2020,
author = {Kevin Yancey and Burr Settles},
publisher = {Harvard Dataverse},
title = {{Replication Data for: A Sleeping, Recovering Bandit Algorithm for Optimizing Recurring Notifications}},
year = {2020},
version = {V1},
doi = {10.7910/DVN/23ZWVI},
url = {https://doi.org/10.7910/DVN/23ZWVI}
}

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/N8XJME
@data{DVN/N8XJME_2017,
author = {Settles, Burr},
publisher = {Harvard Dataverse},
title = {{Replication Data for: A Trainable Spaced Repetition Model for Language Learning}},
year = {2017},
version = {V1},
doi = {10.7910/DVN/N8XJME},
url = {https://doi.org/10.7910/DVN/N8XJME}
}
***

#### Code Will be Updated regularly on Github:
https://github.com/zeineb-ouerghi/Capstone-Project-Senior-Year-/upload/main
